In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Cross Validation  
학습 셋으로 최적의 모델 정확도를 끌어내고 검증 셋으로 과적합을 방지하는 방법을 holdout이라 한다. 그런데 하나의 고정된 검증 셋 데이터로만 모델을 훈련시키면 overfitting이 발생할 가능성은 여전히 존재한다.

### k-Fold Cross Validation  
k-fold cross validation은 전체 학습 데이터 중 일부를 검증 셋으로 분리하는 과정을 k번 반복하는 방법이다.  
![cross-validation](https://user-images.githubusercontent.com/26833433/258589390-8d815058-ece8-48b9-a94e-0e1ab53ea0f6.png)  
일반적으로 k의 수(fold의 수)는 5~10개를 지정하며, k가 클수록 평가의 편중된 정도(bias)는 낮아지지만, 각 fold의 모델별 결과의 분산(variance)이 높을 수 있으므로 k가 무조건 크다고 좋은 것은 아니다.

### LOOCV(Leave-one-out Cross-validation)  
기존 k-fold 방식에서 k를 최대화한 방법이다. 즉 k를 전체 관측치 수로 하여, 검증 셋이 관측치 하나가 되는 것이다. 이 방식은 모델의 편중이 매우 작지만 결과값의 분산이 다소 커 overfitting될 여지가 있다. 또한 데이터의 양이 많은 경우는 적합하지 않다.

### Stratified K-fold Cross Validation  
기존 k-fold 방식에 stratified sampling 방식을 접목한 기법으로, 데이터의 특정 클래스가 한 곳에 몰리는 상황을 방지할 수 있다. Stratified K-fold 교차검증은 분할셋 안의 클래스 비율이 학습셋 전체의 클래스 비율과 같도록 분리해준다.

### Nested Cross Validation
기존 k-fold 방식은 학습셋과 테스트셋 분리를 한번만 하기 때문에, 모델의 성능이 테스트 셋에 크게 의존한다는 문제가 있다. 이러한 문제를 해결하기 위해 학습셋과 테스트셋 검증에도 k-fold 방식을 적용한 것이 Nested CV이다.  
![NestedCV](https://miro.medium.com/v2/resize:fit:1400/1*5vky1z29e1iO6iOvCTBJxg.png)

### Grid Search Cross Validation
Grid Search Cross Validation은 모델의 hyper parameter 값을 리스트 형태로 미리 입력해 놓은 다음 각 조건의 모델 성능을 측정하고 평가하여 효율적으로 최적의 파라미터 값을 찾아내는 기법이다.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, LeaveOneOut, StratifiedKFold
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

In [3]:
# https://www.kaggle.com/datasets/uciml/glass
df = pd.read_csv("../input/glass/glass.csv")

In [4]:
df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [5]:
# k-fold 방법 1

# 종속변수 문자형 변환
df["Type_str"]=df["Type"].apply(str)

# 독립변수, 종속변수 분리
df_x1 = df[['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe']]
df_y1 = df[['Type_str']]

# 기본 모델 설정
rnf_model = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=0)

# 학습셋과 테스트셋 분리하여 생성(6:4)
x_train, x_test, y_train, y_test = train_test_split(
    df_x1,df_y1,test_size=0.4,random_state=10)

# cross_val_score 함수로 k-fold 성능 측정
kfold_scores_1 = cross_val_score(rnf_model, x_train, y_train, cv = 7)

print('k-fold 교차 검증 SCORE : ', kfold_scores_1)
print("k-fold 교차 검증 SCORE 평균 : {:.2f}".format(kfold_scores_1.mean()))

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=7.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel

k-fold 교차 검증 SCORE :  [0.89473684 0.68421053 0.61111111 0.55555556 0.94444444 0.72222222
 0.88888889]
k-fold 교차 검증 SCORE 평균 : 0.76


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


In [6]:
# k-fold 방법 2

# 종속변수 문자형 변환 및 분리
df["Type_str"]=df["Type"].apply(str)
df_x2 = np.array(df[['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe']])
df_y2 = df['Type_str']

# k-fold 설정
kfold = KFold(n_splits=7, shuffle = True, random_state = 37)
kfold_scores_2_mean = []
fold_idx = 0

# k-fold 수행
for train_idx, test_idx in kfold.split(df_x2):
    
    train_x, train_y = df_x2[train_idx], df_y2[train_idx]
    test_x, test_y = df_x2[test_idx], df_y2[test_idx]
    
    # 기본 모델 설정
    rnf_model = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=0)    
    rnf_model.fit(train_x, train_y)
    pred_y = rnf_model.predict(test_x)
    kfold_scores_2 = accuracy_score(test_y, pred_y)    
    fold_idx += 1    
    kfold_scores_2_mean.append(kfold_scores_2)

print(f"k-fold 교차 검증 SCORE 평균 : {np.mean(kfold_scores_2_mean)}")

k-fold 교차 검증 SCORE 평균 : 0.7526881720430108


In [7]:
# LOOCV (Leave-one-out Cross-validation)

# 기본 모델 설정
rnf_model = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=0)
rnf_model.fit(train_x, train_y)
pred_y = rnf_model.predict(test_x)

loocv = LeaveOneOut()
loocv_scores = cross_val_score(rnf_model,test_x,test_y, cv = loocv)

print("테스트 셋 전체 관측치 수 : ", len(test_y))
print("LOOCV 검증 분할 횟수 : ", len(loocv_scores))
print("LOOCV 교차 검증 SCORE 평균 : {:.2f}".format(loocv_scores.mean()))

테스트 셋 전체 관측치 수 :  30
LOOCV 검증 분할 횟수 :  30
LOOCV 교차 검증 SCORE 평균 : 0.70


In [9]:
# Stratified K-fold Cross Validation

# Stratified K-fold 교차검증 설정
strati = StratifiedKFold(n_splits=3)

n_iter=0
print("전체 데이터셋 범주 별 관측치 수 :\n",df_y2.value_counts())
for train_index, test_index in strati.split(df_x2,df_y2):
    n_iter +=1
    strati_train_y = df_y2.iloc[train_index]
    strati_test_y = df_y2.iloc[test_index]
    print('분할 {0}'.format(n_iter))
    print('학습 셋 범주 별 관측치 수:\n', strati_train_y.value_counts())
    print('검증 셋 범주 별 관측치 수:\n', strati_test_y.value_counts())

strati_scores = cross_val_score(rnf_model,test_x,test_y, cv = strati)
print("Stratified K-fold 교차 검증 SCORE 평균 : {:.2f}".format(strati_scores.mean()))

전체 데이터셋 범주 별 관측치 수 :
 Type_str
2    76
1    70
7    29
3    17
5    13
6     9
Name: count, dtype: int64
분할 1
학습 셋 범주 별 관측치 수:
 Type_str
2    51
1    46
7    19
3    11
5     9
6     6
Name: count, dtype: int64
검증 셋 범주 별 관측치 수:
 Type_str
2    25
1    24
7    10
3     6
5     4
6     3
Name: count, dtype: int64
분할 2
학습 셋 범주 별 관측치 수:
 Type_str
2    50
1    47
7    20
3    12
5     8
6     6
Name: count, dtype: int64
검증 셋 범주 별 관측치 수:
 Type_str
2    26
1    23
7     9
3     5
5     5
6     3
Name: count, dtype: int64
분할 3
학습 셋 범주 별 관측치 수:
 Type_str
2    51
1    47
7    19
3    11
5     9
6     6
Name: count, dtype: int64
검증 셋 범주 별 관측치 수:
 Type_str
2    25
1    23
7    10
3     6
5     4
6     3
Name: count, dtype: int64


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Stratified K-fold 교차 검증 SCORE 평균 : 0.67


In [10]:
# Nested Cross Validation 및 Grid Search Cross Validation

# 학습셋과 테스트셋 분리하여 생성(6:4)
x_train, x_test, y_train, y_test = train_test_split(
    df_x2,df_y2,test_size=0.4,random_state=10)

# 모델 시도 횟수 설정
NUM_TRIALS = 20

# 그리드 하이퍼 파라미터 설정
hp_para = {'max_depth':[2,3,4], 'min_samples_split':[2,3]}

# 스코어를 array로 저장
nested_scores = np.zeros(NUM_TRIALS)

# 교차 검증 반복 수행
for i in range(NUM_TRIALS):

    inner_loop = KFold(n_splits=3, shuffle=True, random_state=i)
    outer_loop = KFold(n_splits=3, shuffle=True, random_state=i)

    # Nested Cross Validation 하이퍼 파라미터 최적화
    gs_cv = GridSearchCV(rnf_model, param_grid=hp_para, cv=inner_loop)
    nested_score = cross_val_score(gs_cv, X=x_train, y=y_train, cv=outer_loop)
    nested_scores[i] = nested_score.mean()

# 테스트셋에 모델 적용
gs_cv.fit(x_test, y_test)

print("각 TRIAL 별 SCORE 평균 : \n", nested_scores)
print("전체 TRIAL SCORE 평균 : {:.2f}".format(nested_scores.mean()))
print("최대 TRIAL SCORE : {:.2f}".format(nested_scores.max()))
print("최적 하이퍼 파라미터 : \n", gs_cv.best_params_)

각 TRIAL 별 SCORE 평균 : 
 [0.71059432 0.71077889 0.67884828 0.72646733 0.71834625 0.69545958
 0.72683647 0.68752307 0.6720192  0.68014027 0.72757475 0.7113326
 0.68014027 0.7113326  0.66426726 0.68807678 0.73366556 0.73421927
 0.67109635 0.70302695]
전체 TRIAL SCORE 평균 : 0.70
최대 TRIAL SCORE : 0.73
최적 하이퍼 파라미터 : 
 {'max_depth': 4, 'min_samples_split': 2}
